### A Binary Classifier built using a FC network, with PyTorch

We will be working with the Attrition dataset and we want to build a binary classifier: a model enabling to predict if the employee wants to leave the company (target = 1)

In this Notebook we will show how-to implement the same model we have seen in chap. 3, using PyTorct Lightning

Here we explore:
* how to define Network Architecture
* How to write the training loop

We can see that less code is needed when using PyTorch Lightning

In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import optim
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

from attrition_dataset import AttritionDataset

# pyplot to plot the training loss
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# check that access to GPU is OK
torch.cuda.is_available()

True

In [3]:
# need to make training reproducible
torch.manual_seed(42)

In [5]:
# globals

LEARNING_RATE = 0.001
EPOCHS = 800

#### Here you find defined the Network architecture

In [6]:
# we need to implement a class subclassing nn.Module, writing the forward method

#non conviene avere una rete troppo profonda e con troppi parametri... i dati non sono tantissimi
class FirstFC(pl.LightningModule):
    # here we define the structure
    def __init__(self):
        super(FirstFC, self).__init__()
        self.fc1 = nn.Linear(30, 30)
        self.fc2 = nn.Linear(30, 30)
        self.fc3 = nn.Linear(30, 30)
        self.fc4 = nn.Linear(30, 10)
        self.fc5 = nn.Linear(10, 1)
        # fondamentale memorizzare anche il dropout tra gli attributi 
        # per disattivarlo in modalità eval
        # altrimenti in eval i risultati non sono pienamente riproducibili
        self.drop = nn.Dropout(0.1)
    
    # here we implement the forward pass
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.drop(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        x = self.drop(x)
        
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        
        # the last layer must give the prob that it is positive... so we use sigmoid
        # so we must use nn.BCELoss
        x = torch.sigmoid(self.fc5(x))
        
        return x
    
    def configure_optimizers(self):
        # qui possiamo anche pluggare un LR scheduler !!!
        optimizer = optim.Adam(self.parameters(), lr=LEARNING_RATE)
        
        return optimizer
    
    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        
        outputs = self.forward(inputs)
            
        # we need to add this !
        targets = targets.unsqueeze(1)
            
        loss = nn.BCELoss()(outputs, targets)
        
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        
        outputs = self.forward(inputs)
            
        # we need to add this !
        targets = targets.unsqueeze(1)
            
        loss = nn.BCELoss()(outputs, targets)
        
        self.log("validation_loss", loss, on_epoch=True, prog_bar=True)
        
        return loss

In [7]:
# dataset
attrition_path = "/opt/notebooks/ads-examples/oracle_data/orcl_attrition.csv"

attrition_ds = AttritionDataset(attrition_path, over_sample=True)

# split in train and validation
FRAC = 0.1
N_TRAIN = int(len(attrition_ds)*(1. - FRAC))
N_VALID = len(attrition_ds) - N_TRAIN

train_set, val_set = torch.utils.data.random_split(attrition_ds, [N_TRAIN, N_VALID], generator=torch.Generator().manual_seed(42))

print(f"We have {len(train_set)} samples in training set...")
print(f"We have {len(val_set)} samples in validation set...")

We have 2219 samples in training set...
We have 247 samples in validation set...


In [8]:
# oversampling the size has greatly increased performance

In [9]:
# dataloaders

# aumentando il btch size le prestazioni peggiorano
BATCH_SIZE = 64

train_dl = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=6)
val_dl = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=6)

#### Do the training

In [10]:
%%time
# istantiate the Lightning model
model = FirstFC()

trainer = pl.Trainer(max_epochs=EPOCHS, log_every_n_steps=35, accelerator="gpu", devices=1)

trainer.fit(model, train_dl, val_dl)

print()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | fc1  | Linear  | 930   
1 | fc2  | Linear  | 930   
2 | fc3  | Linear  | 930   
3 | fc4  | Linear  | 310   
4 | fc5  | Linear  | 11    
5 | drop | Dropout | 0     
---------------------------------
3.1 K     Trainable params
0         Non-trainable params
3.1 K     Total params
0.012     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=800` reached.



CPU times: user 5min 59s, sys: 5min 24s, total: 11min 24s
Wall time: 11min 46s


### evaluate the model and compute accuracy

In [15]:
# è fondamentale mettere il modello in modalità eval
# per disabilitare il dropout che altrimenti produrrebbe un certo
# livello di casualità nell'inferenza

model.eval()

device = "cpu"

THR = 0.5

tot_correct = 0

for batch in val_dl:
    inputs, targets = batch
    
    inputs = inputs.to(device)
    targets = targets.to(device)
        
    outputs = model(inputs)
    
    class_outputs = torch.where(outputs >= THR, 1, 0)
    class_outputs = class_outputs.squeeze(1)
    outputs = outputs.squeeze(1)
    
    is_ok = torch.eq(class_outputs, targets)
    
    # ora è suffciente sommare True = 1
    tot_correct += is_ok.sum().item()

accuracy = round(tot_correct/len(val_set), 3)

print()
print("Summary:")
print(f"Accuracy on validation set is: {accuracy}")


Summary:
Accuracy on validation set is: 0.725


### conclusioni:
* over-sampling (fatto dalla classe AttritionDataset
* una definizione attenta degli iper-parametri (faticosa, n. neuroni, LR)

consentono di ottenere un'accuratezza tra il 73% e 80%... con TabNet vedremo che si riesce ad arrivare al 95% 